In [69]:
from web3 import Web3
from solcx import install_solc, compile_standard
import solcx
import json
import os

install_solc(version='0.8.0')

onchain = Web3(Web3.HTTPProvider('http://localhost:8545')) #on-chain

s1 = Web3(Web3.HTTPProvider('http://localhost:8546')) #shard 1
s2 = Web3(Web3.HTTPProvider('http://localhost:8547')) #shard 2
s3 = Web3(Web3.HTTPProvider('http://localhost:8548')) #shard 3

onchain.isConnected(), s1.isConnected(), s2.isConnected(), s3.isConnected()

(True, True, True, True)

In [70]:
# DEPLOY DELL'ON CHAIN MANAGER

path_on_chain = "prova_on_chain.sol"
# Leggo il file contenente lo smart contract
with open(path_on_chain, "r") as file:
    text_file = file.read()

solcx.install_solc('0.8.0')

# Compilo il file contenente lo smart contract
compiled_sol = compile_standard(
        {
            "language": "Solidity",
            "sources": {"prova_on_chain.sol": {"content": text_file}},
            "settings": {
                "outputSelection": {
                    "*": {
                        "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"] # output needed to interact with and deploy contract 
                    }
                }
            },
        },
        solc_version='0.8.0',
    )

# get bytecodes
bytecode = compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["metadata"])["output"]["abi"]

# seleziono l'account di default
# DA MODIFICARE METTENDO GLI ACCOUNT INSERITI DALL'UTENTE
onchain.eth.default_account = onchain.eth.accounts[0]
s1.eth.default_account = s1.eth.accounts[0]
s2.eth.default_account = s2.eth.accounts[0]
s3.eth.default_account = s3.eth.accounts[0]

# deploy dello smart contract dell'onchain manager
On_Chain_Manager = onchain.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = On_Chain_Manager.constructor().transact()
tx_receipt = onchain.eth.wait_for_transaction_receipt(tx_hash)

# creazione dell'oggetto contract per lo smart contract dell'onchain manager in modo da poterci interagire
on_chain_manager = onchain.eth.contract(
     address=tx_receipt.contractAddress,
     abi=abi
)

In [71]:
# codice per far inserire all'utente il path dello smart contract da deployare

print("Inserisci la path del file solidity dello smart contract di cui fare il deploy")
file_path = input()

# controllo che il file esista effettivamente e che sia di tipo .sol
while not os.path.exists(file_path) and os.path.splitext(file_path)[1] != ".sol":
    os.system('clear')
    print("La path deve esistere e il file deve essere di tipo .sol")
    file_path = input()
    
# Leggo il file contenente lo smart contract
with open(file_path, "r") as file:
    text_file = file.read()

Inserisci la path del file solidity dello smart contract di cui fare il deploy


In [72]:
# codice per ottenere solo con il nome del file
import ntpath
file_name = ntpath.basename(file_path)
print(file_name)

StorageFactory.sol


In [73]:
# codice per leggere la versione di solidity
import re

# espressione regolare utilizzata per trovare dentro allo smart contract la versione di solidity richiesta
regular_expression = "pragma solidity .*;"
# ricerca della stringa pragma dentro lo mart contract tramite espressione regolare
match = re.search(regular_expression, text_file)
# caratteri da rimuovere per rimanere con la stringa composta solo dalla versione di solidity
chars_to_rempove = ['>', '^', '=', ';']
# rimozione dei caratteri
text_version = match.group().split()[2].translate(str.maketrans('','',''.join(chars_to_rempove)))

print(text_version)


0.6.0


In [74]:
# codice per leggere gli import
import re

# espressione regolare utilizzata per trovare dentro allo smart contract la versione di solidity richiesta
regular_expression = "import .*;"
# ricerca della stringa pragma dentro lo mart contract tramite espressione regolare
match = re.findall(regular_expression, text_file)
# caratteri da rimuovere per rimanere con la stringa composta solo dalla versione di solidity
chars_to_rempove = [';', '"']
# rimozione dei caratteri
imports = []
for item in match:
    import_text = item.split()[1]
    import_text = import_text.translate(str.maketrans('','',''.join(chars_to_rempove)))
    imports.append(import_text)

print(imports)

['/Users/andreacivitarese/VS_projects/ProgettoSSB/2022/SimpleStorage.sol']


In [77]:
# AGGIUNGERE IL TRY EXCEPT PER GLI ERRORI DI COMPILAZIONE

# Installo la versione del compilatore richiesta dallo smart contract
solcx.install_solc(text_version)

#compilo il file
compiled_solidity = solcx.compile_files([file_path], solc_version=text_version)

# estraggo i bytecode degli smart contract compilati
bytecodes = []
for key in compiled_solidity.keys():
    bytecodes.append(compiled_solidity[key]['bin'])

# estraggo le abi degli smart contract
abis = []
for key in compiled_solidity.keys():
    abis.append(json.loads(json.dumps(compiled_solidity[key]))['abi'])


In [82]:
#QUESTO BLOCCO NON SERVE SE VIENE ESEGUITO QUELLO SOPRA
#try:
solcx.install_solc(text_version)
    # Compilo il file contenente lo smart contract
compiled_sol = compile_standard(
        {
            "language": "Solidity",
            "sources": {file_name: {"content": text_file}},
            "settings": {
                "outputSelection": {
                    "*": {
                        "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"] # output needed to interact with and deploy contract 
                    }
                }
            },
        },
        solc_version=text_version,
        allow_paths=imports
    )

    # get bytecodes
bytecodes = []
for item in compiled_sol["contracts"][file_name].keys():
    bytecodes.append(compiled_sol["contracts"][file_name][item]["evm"]["bytecode"]["object"])   

    # get abi
abis = []
for item in compiled_sol["contracts"][file_name].keys(): 
    abis.append(json.loads(compiled_sol["contracts"][file_name][item]["metadata"])["output"]["abi"])

print(abis)
#except:
#    os.system('clear')
#    print("La compilazione dello smart contract non è andata a buon fine")

[[{'inputs': [], 'name': 'createSimpleStorageContract', 'outputs': [], 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [{'internalType': 'uint256', 'name': '_simpleStorageIndex', 'type': 'uint256'}], 'name': 'sfGet', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'stateMutability': 'view', 'type': 'function'}, {'inputs': [{'internalType': 'uint256', 'name': '_simpleStorageIndex', 'type': 'uint256'}, {'internalType': 'uint256', 'name': '_simpleStorageNumber', 'type': 'uint256'}], 'name': 'sfStore', 'outputs': [], 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'name': 'simpleStorageArray', 'outputs': [{'internalType': 'contract SimpleStorage', 'name': '', 'type': 'address'}], 'stateMutability': 'view', 'type': 'function'}]]


In [86]:
# richiesta all'on-chain-manager su quale shard deployare lo smart contract
shard_where_deploy = on_chain_manager.functions.get_shard_where_deploy().call()
print(shard_where_deploy)

if shard_where_deploy == 1:
     blockchain_where_deploy = s1
elif shard_where_deploy == 2:
     blockchain_where_deploy = s2
elif shard_where_deploy == 3:
     blockchain_where_deploy = s3


# esecuzione della transazione di deploy dello smart contract
contratto = blockchain_where_deploy.eth.contract(abi=abis[0], bytecode=bytecodes[0])
tx_hash = contratto.constructor().transact()
tx_receipt = blockchain_where_deploy.eth.wait_for_transaction_receipt(tx_hash)

# creazione dell'oggetto contratto per interagirci
contratto_deployato = blockchain_where_deploy.eth.contract(
     address=tx_receipt.contractAddress,
     abi=abi
)

print(contratto_deployato.address)


1
0x4C3c212D2C1aeF772Bff6b3256C6c2f9064bf2fd


In [89]:
# Registro che il contratto è stato deployato sull'onchain manager
on_chain_manager.functions.register_contract(contratto_deployato.address, shard_where_deploy).transact()

# Test per vedere se il prossimo smart contract viene detto di deployarlo sulla shard dopo
print(on_chain_manager.functions.get_shard_where_deploy().call())

2
